# Klasyfikacja oparta o n-gramowe modele językowe

Ten plik pozwala na seryjne testowanie klasyfikatorów i pozyskanie wyników (w formacie tabeli latexowych, macierzy pomyłek) dla określonych zbiorów zapisanych w formacie pickle, a także na seryjne tworzenie tworzenie własnych zbiorów i zapisywanie ich jako pickle. Uruchomienie wszystkich komórek z notatnika powoduje wygenerowanie wyników dla wszystkich okreśonych zbiorów zapamiętanych w formacie pickle. Klasyfikator korzysta z plików ARPA z katalogu ../arpa_files/arpa/ (uwaga: zapamiętane modele to modele 6-grmowe), a narzędzie lmplz z kodu biblioteki kenlm, której pliki mieszczą się w katalogu ../../kenlm (należy zainstalować tę bibliotekę, korzystając z instrukcji z README.md).

### Działania przygotowawcze: podłączenie Dysku Google, instalacja wymaganych pakietów, dołączenie katalogu projektu do ścieżki systemowej i import potrzebnych klas z projektu

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
# ścieżka do bieżącego katalogu (należy edytować, jeśli się nie zgadza)
%cd gdrive/MyDrive/praca_inzynierska/authorship_analysis_project/training_models

In [ ]:
! pip install -r ../requrements.txt
! pip install https://github.com/kpu/kenlm/archive/master.zip
! python -m spacy download pl_core_news_lg

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
print(module_path)
print(sys.path)

Narzędzie lmplz służy to tworzenia nowych plików w formacie ARPA.

In [ ]:
! chmod 777 ../../kenlm/build/bin/lmplz
! ls -al ../../kenlm/build/bin/lmplz

In [ ]:
import numpy as np

from data_preparation import CorpusPreparation, ArpaPreparation
from pipelines import Pipeline, Explain
from data_fetchers import EpochsFetcher, BookSet

### Zbiory testowe

Lista dostępnych zbiorów zapisanych jako pickle. Zbiory oznaczone są liczbami naturalnymi i cyfra dziesiątek oznacza sposób podziału zbioru testowego z określoną liczbą słów w próbce, a cyfra jedności oznacza wybrany preprocessing. Informacje o zbiorze zawierają liczbę słów, na podstawie których został utworzony model językowy dla każdego autora (plik ARPA)

In [ ]:
import pickle
from os.path import exists
datasets_filepath = 'datasets/ngram_'
for i in range(100):
  path = datasets_filepath + str(i)
  if exists(path):
    print(path)
    with open(path, 'rb') as f:
      data = pickle.load(f)
      head_data, _, _ = data
      print(f'test_size: {head_data[0]}, train_size: {head_data[1]}, number of words in paragraph: {head_data[2]}, preprocessing operations: {head_data[3]}, authors: {head_data[4]}')

datasets/ngram_0
test_size: 20, train_size: 1500, number of words in paragraph: 15, preprocessing operations: ['anonymize'], authors: ['Daniel Naborowski', 'Mikołaj Sęp Szarzyński', 'Elżbieta Drużbacka', 'Adam Mickiewicz', 'Juliusz Słowacki', 'Cyprian Kamil Norwid', 'Adam Asnyk', 'Maria Konopnicka', 'Kazimierz Przerwa-Tetmajer', 'Bolesław Leśmian', 'Jan Kasprowicz']
datasets/ngram_1
test_size: 20, train_size: 1500, number of words in paragraph: 15, preprocessing operations: ['anonymize', 'lower_text', 'remove_stop_words'], authors: ['Daniel Naborowski', 'Mikołaj Sęp Szarzyński', 'Elżbieta Drużbacka', 'Adam Mickiewicz', 'Juliusz Słowacki', 'Cyprian Kamil Norwid', 'Adam Asnyk', 'Maria Konopnicka', 'Kazimierz Przerwa-Tetmajer', 'Bolesław Leśmian', 'Jan Kasprowicz']
datasets/ngram_2
test_size: 20, train_size: 1500, number of words in paragraph: 15, preprocessing operations: ['anonymize', 'lower_text', 'POS_leave_only'], authors: ['Daniel Naborowski', 'Mikołaj Sęp Szarzyński', 'Elżbieta Dru

### Określenie rodzajów preprocessingu i wybór zbiorów, deklaracja zmiennych

In [ ]:
preps_num = [0,1,2,3,5] # wybór preprocessingu
preps_descr = ['a', 'als', 'alo_NVAdj', 'alp', 'alsp', 'alomp_NVAdj']
preps_lists = [
    ['anonymize'],
    ['anonymize', 'lower_text', 'remove_stop_words'],
    ['anonymize', 'lower_text','POS_leave_only'],
    ['anonymize', 'lower_text', 'remove_punctuation'],
    ['anonymize', 'lower_text', 'remove_stop_words', 'remove_punctuation'],
    ['anonymize', 'lower_text','POS_leave_only', 'lemmatize_text', 'remove_punctuation']
]
pos = ['ADJ', 'NOUN', 'VERB']
test_sizes = [(15,20), (30, 10), (50, 40), (100, 20), (30, 60), (30, 400), (300, 400), (1000, 120)]
save_dataset = False
n_model = 4

div_num= [5,6,7] # 0,1,2,3,4,5,6,7 # wybór numeru podziału zbioru z wielkością próbki
train_size_words = 300000 # 1500, 10000, 300000
dataset = 'X' # 'Y' -- L10 / 'Y_p' -- L6 / 'X' -- E3

mixed = None
test_size = None
train_size = None
words_num_in_par = None
representations = ['empty']
models= ['ng']
authors = []
preprocessing_list = []
train_classes = []
prep_descr = ''
kinds_descr = ''
latex_set_str = ''


### Definicje funkcji

In [ ]:
def get_books_epoch_list():
  bookset = BookSet()
  bookset.fetch()
  if dataset == 'X':
    kinds = ['Epika']
    kinds_descr ='e'
    authors = ['Eliza Orzeszkowa', 'Henryk Sienkiewicz', 'Bolesław Prus']
    books_list = bookset.list_of_books_to_author_title_list(bookset.get_books_by_authors_list_kinds(authors, kinds))
    books_epoch_list = [books_list]
  if dataset =='Y':
    kinds = ['Liryka']
    kinds_descr = 'l'
    authors = ['Mikołaj Sęp Szarzyński', 'Elżbieta Drużbacka', 'Adam Mickiewicz', 'Juliusz Słowacki', 'Cyprian Kamil Norwid', 'Adam Asnyk', 'Maria Konopnicka', 'Kazimierz Przerwa-Tetmajer', 'Bolesław Leśmian', 'Jan Kasprowicz']
    authors = ['Daniel Naborowski', 'Mikołaj Sęp Szarzyński', 'Elżbieta Drużbacka', 'Adam Mickiewicz', 'Juliusz Słowacki', 'Cyprian Kamil Norwid', 'Adam Asnyk', 'Maria Konopnicka', 'Kazimierz Przerwa-Tetmajer', 'Bolesław Leśmian', 'Jan Kasprowicz']
    authors_b = ['Mikołaj Sęp Szarzyński', 'Elżbieta Drużbacka']
    authors_r = ['Adam Mickiewicz', 'Juliusz Słowacki', 'Cyprian Kamil Norwid']
    authors_p = ['Adam Asnyk', 'Maria Konopnicka']
    authors_m = ['Kazimierz Przerwa-Tetmajer', 'Bolesław Leśmian', 'Jan Kasprowicz']
    books_list_b = bookset.list_of_books_to_author_title_list(bookset.get_books_by_authors_list_kinds(authors_b, kinds))
    books_list_r = bookset.list_of_books_to_author_title_list(bookset.get_books_by_authors_list_kinds(authors_r, kinds))
    books_list_p = bookset.list_of_books_to_author_title_list(bookset.get_books_by_authors_list_kinds(authors_p, kinds))
    books_list_m = bookset.list_of_books_to_author_title_list(bookset.get_books_by_authors_list_kinds(authors_m, kinds))
    books_epoch_list = [books_list_b, books_list_r, books_list_p, books_list_m]
  if dataset =='Y_p':
    kinds = ['Liryka']
    kinds_descr = 'l'
    authors = ['Adam Mickiewicz', 'Juliusz Słowacki', 'Maria Konopnicka', 'Kazimierz Przerwa-Tetmajer', 'Bolesław Leśmian', 'Jan Kasprowicz']
    authors = ['Daniel Naborowski', 'Mikołaj Sęp Szarzyński', 'Elżbieta Drużbacka', 'Adam Mickiewicz', 'Juliusz Słowacki', 'Cyprian Kamil Norwid', 'Adam Asnyk', 'Maria Konopnicka', 'Kazimierz Przerwa-Tetmajer', 'Bolesław Leśmian', 'Jan Kasprowicz']
    authors_r = ['Adam Mickiewicz', 'Juliusz Słowacki']
    authors_p = ['Maria Konopnicka']
    authors_m = ['Kazimierz Przerwa-Tetmajer', 'Bolesław Leśmian', 'Jan Kasprowicz']
    books_list_r = bookset.list_of_books_to_author_title_list(bookset.get_books_by_authors_list_kinds(authors_r, kinds))
    books_list_p = bookset.list_of_books_to_author_title_list(bookset.get_books_by_authors_list_kinds(authors_p, kinds))
    books_list_m = bookset.list_of_books_to_author_title_list(bookset.get_books_by_authors_list_kinds(authors_m, kinds))
    books_epoch_list = [books_list_r, books_list_p, books_list_m]
  print(books_epoch_list)
  return books_epoch_list, authors, kinds_descr

In [ ]:
def get_existing_authors():
  existing_authors_set = set(test_classes)
  print(existing_authors_set)
  authors_max_books = {}
  authors_ = []
  books_epoch_list, _, _ = get_books_epoch_list()
  #print(books_epoch_list)
  for books_epoch in books_epoch_list:
      authors_list = [author for (author, title) in books_epoch]
      authors_set = sorted(list(set(authors_list)))
      for author in authors_set:
          if author in existing_authors_set:
              books_num = authors_list.count(author)
              if author in authors_:
                  if books_num > authors_max_books[author]:
                      authors_.remove(author)
                      authors_max_books[author] = books_num
                      authors_.append(author)
              else:
                  authors_.append(author)
                  authors_max_books[author] = books_num
  return authors_

In [ ]:
def conf_path():
  div = ''
  if not mixed:
      div = '_div'
  path = f'../figures/ng4_x_fig{train_size}_{test_size}_{words_num_in_par}{div}_{prep_descr}_{kinds_descr}.jpg'
  return path

def get_latex_set_str(p: int, d: int):
  prep = preps_descr[p].partition('_')[0].upper()
  if d < 2:
    st = 'Y'
  elif d < 5:
    st = "Y'"
  else:
    st = 'X'
  return '$' + chr(92) + 'mathbb{' + st + '}_{' + prep + '}$'

In [ ]:
def run_pipeline(pr, train_size_words):
  authors_ = get_existing_authors()
  print(authors_)
  prep = preps_descr[pr]
  p = Pipeline(
    corpus_train=np.array([prep, train_size_words]),
    corpus_test=test_set,
    classes_train=test_classes,
    classes_test=test_classes,
    class_names=authors_,
    representations=representations,
    models=models)
  p.pipelines()
  p.accuracy_latex_format('../results/ng4_x_table.tex', latex_set_str)
  p.save_img(conf_path())
  #expl = Explain(p)

### Tworzenie nowych modeli n-gramowych i zbiorów testowych

In [ ]:
if save_dataset:
  train_corpuses_dir = '../arpa_files/train_corpuses'
  arpa_dir = '../arpa_files/arpa'
  for p in preps_num:
    books_epoch_list, authors, kinds_descr = get_books_epoch_list()
    preprocessing_list = preps_lists[p]
    ap = ArpaPreparation()
    ap.read_corpus_form_frozen_dataset(authors, preps_descr[p], kinds_descr)
    #ap.get_books()
    #ap.books_preprocessing(preprocessing_list, pos)
    for file_ in os.listdir(train_corpuses_dir):
      file_ = file_.replace(" ", "\ ")
      file_corpus = train_corpuses_dir+'/'+file_
      ! rm $file_corpus
    ap.create_train_txt_corpus_for_arpa(train_size_words)
    for file in os.listdir(train_corpuses_dir):
      print(file)
      file = file.replace(" ", "\ ")
      file_arpa = arpa_dir + '/'+file[:-4]+'_'+preps_descr[p]+str(train_size_words)+'.arpa'
      file_corpus = train_corpuses_dir+'/'+file
      ! echo $file
      ! ../../kenlm/build/bin/lmplz -o $n_model --discount_fallback <$file_corpus >$file_arpa
      for d in div_num:
        number_of_dataset = str(d)+str(p) if d>0 else str(p)
        words_num_in_par, test_size = test_sizes[d]
        test_set, test_classes = ap.get_test_corpus(test_size, words_num_in_par)
        print(words_num_in_par)
        file_name_save = datasets_filepath+number_of_dataset
        with open(file_name_save, 'wb') as f:
          head_data = [test_size, train_size_words, words_num_in_par, preprocessing_list, authors]
          data = [head_data, test_set, test_classes]
          pickle.dump(data, f)

### Pętla klasyfikacji

In [ ]:
if not save_dataset:
  for p in preps_num:
    for d in div_num:
      number_of_dataset = str(d)+str(p) if d>0 else str(p)
      if d < 5:
        mixed = True
        kinds_descr = 'l'
      else:
        mixed = False
        kinds_descr = 'e'
      latex_set_str = get_latex_set_str(p,d)
      prep_descr = preps_descr[p]
      file_name_read = datasets_filepath + str(number_of_dataset)
      with open(file_name_read, 'rb') as f:
        data = pickle.load(f)
        head_data, test_set, test_classes = data
        test_size, train_size_words, words_num_in_par, preprocessing_list, authors = head_data
      print(get_books_epoch_list())
      run_pipeline(p, train_size_words)